# how to use pyfetch 사용설명서

## 작동원리
- requests라는 라이브러리를 이용하여 서버에 데이터를 요청한다.

### requests 설치
.py 파일에서 설치하려면 콘솔을 열고 !없이 

```pip install requests```

를 입력(버전 상관 없음)

In [1]:
!pip install requests

### requests 라이브러리 설치 확인
버전이 출력되면 제대로 설치된것임

In [3]:
import requests

print(requests.__version__)

2.28.1


### url과 헤더 설명

#### url=http://(실행하는 서버 ip주소):(포트번호)

  - 포트번호는 지난번 기본설정값 3000으로 하면 됨. 만약 서버 포트번호 바뀌면 바뀐 번호로 써주면 됨.

  - 그리고 ip주소는 만약 같은 컴퓨터에서 실행한다면
  
    ```url=http://127.0.0.1:3000```

    로 하면 될것임.

    

  - 만약 다른 컴퓨터에서 서버를 실행한다면?

    지난번에 알려준 서버 ip주소 얻는법에서 서버 주소가```192.168.0.3```이 나왔다면

    ```url=http://192.168.0.3:3000```

    으로 작성하변 됨

  

#### 헤더

``` headers = {'Content-Type': 'application/json','Authorization': 'Bearer TOKEN',} ```


#### url과 headers변수 입력

In [4]:
url='http://192.168.0.4'

headers = {'Content-Type': 'application/json','Authorization': 'Bearer TOKEN',}

#### 실제 요청 전송

#### url 변수 변경, identification추가
다 post로 처리하도록 서버를 설정해놓음(원래는 이러면 안됨.)

그리고 적절하게 요청을 하기 위해서는 url을 조금 바꿔야함.

추가로 인증과정을 서버에 둬서 서버에 인증을 위한 데이터를 첨부해줘야함.

In [16]:
# 바꾼 url
url = 'http://192.168.0.4:3000/medical_staff/requestCounselsPython'

# 인증을 위한 identification 데이터
# email과 password 변수에 의료진 email과 password 문자열로 적으면 됨
email = 'medical1'
password = '1234'
identification = {'email':email, 'password':password}

# 최종적으로 서버에 보낼 데이터
sendData = { 'identification':identification }

#### 요청하기

결과적으로 아래 코드에서 resData에 json형식으로 변수가 저장되어 있음

resData 읽어보고 적절하게 필요한 데이터 가져다 쓰면 될것임

In [19]:
response = requests.post(url, headers=headers, json=sendData)

resData = response.json()

# 오류 없음
if resData['result'] == 0:
    print(resData)
elif resData['result'] == 1:
    print('로그인 에러. email이나 Password 변수 점검 바람.')
    print(resData)
elif resData['result'] == 2:
    print('db서버로부터 서버가 파일을 읽어오는데 오류가 있거나 서버가 .json 파일을 읽어오는데 오류가 발생함')
    print(resData)

{'result': 0, 'data': {'counselList': [{'id': 1, 'path': '/Users/m1_macair/Desktop/외주/analyze_with_photo/analyze_with_photo/analyze_with_photo_server/counsels/1723517940503.json', 'isDiagnosed': False, 'createdAt': '2024-08-13T02:59:00.505Z', 'updatedAt': '2024-08-13T02:59:00.505Z', 'deletedAt': None, 'UserId': None, 'userId': 1}, {'id': 2, 'path': '/Users/m1_macair/Desktop/외주/analyze_with_photo/analyze_with_photo/analyze_with_photo_server/counsels/1723517985099.json', 'isDiagnosed': True, 'createdAt': '2024-08-13T02:59:45.202Z', 'updatedAt': '2024-08-13T04:47:43.808Z', 'deletedAt': None, 'UserId': None, 'userId': 1}]}}


### 전체 코드

위에서 설명이 길었는데 그냥 아래 코드 복붙하고 실행시키면 동일 폴더에 temp.json생김. 여기서 가져다 쓰면 됨.

이때 
- url내부에 ip주소
- email과 password 아무 의료진 이메일 패스워드(회원가입 의료진으로 체크한 아무 계정이나 상관없음)
만 바꿔주면 됨

그리고 오류 발생 없으면 temp.json 내부에 있는 값 가져다 쓰면 됨.

오류 발생하면 어떤 print된값 캡쳐해서 나한테 보내주면 뭔오륜지 알려드림

In [45]:
import requests

headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer TOKEN',
}

# 바꾼 url
url = 'http://192.168.0.4:3000/medical_staff/requestCounselsPython'

# 인증을 위한 identification 데이터
# email과 password 변수에 의료진 email과 password 문자열로 적으면 됨
email = 'medical1'
password = '1234'
identification = {'email':email, 'password':password}

# 최종적으로 서버에 보낼 데이터(사용자 인증을 위해서)
sendData = { 'identification':identification }

# 서버에 데이터 요청
response = requests.post(url, headers=headers, json=sendData, stream=True)

# 서버에서 받은 데이터 json으로 파싱
resData = response.json()

with open('./temp.json', 'wb') as f:
    for chunk in response.iter_content(chunk_size=8192):  # 8KB씩 청크로 읽기
        if chunk:  # 청크가 비어있지 않다면
            f.write(chunk)

# 정상작동시
if resData['result'] == 0:
    print('정상작동')
# 로그인 에러 발생시
elif resData['result'] == 1:
    print('로그인 에러. email이나 Password 변수 점검 바람.')
    print(resData)
# 서버에서 에러 발생시
elif resData['result'] == 2:
    print('db서버로부터 서버가 파일을 읽어오는데 오류가 있거나 서버가 .json 파일을 읽어오는데 오류가 발생함')
    print(resData)


정상작동


### 참고

#### temp.json 형식

```
{
    'result': 0,(오류발생여부, 0은 정상작동, 1은 로그인 실패, 2는 서버 오류) 
    'data': {
        'counselList':
            # 아래 배열은 councelList 배열임.
            [
                {
                    'id': 1,
                    'counsel': (각 id의 대화내역, 사진 저장을 base64형식으로 해서 사진 첨부할 경우 엄청 길수도 있음),
                    'isDiagnosed': False,(진단받았는지 여부)
                    'createdAt': '2024-08-13T02:59:00.505Z', 
                    'updatedAt': '2024-08-13T02:59:00.505Z',
                    'deletedAt': None,
                     'UserId': None,(신경안써도 됨)
                     'userId': 1
                }, 
                (위 json 형식이 배열로 있음)
            ]
    }
}
```

#### counsel json 형식

temp_altered.json 파일 참고


## 보너스: json파일에 사진 데이터 사진파일로 변환하고 사진파일 데이터는 삭제

위의 과정에서 서버로부터 받아온 temp.json 파일의 \
사진데이터를 'output(counsel의 id).jpg'로 변환하여 저장하고\
사진데이터를 temp.json파일에서 삭제한뒤\
(정확히는 사진데이터가 있던 자리에 'photo was here'같은 문자열로 대체해서 용량을 줄임)\
temp_altered.json파일로 저장함.

그냥 아래 코드도 같이 쓰고, temp_altered.json으로 데이터 사용한다고 생각하면 편함

In [55]:
import json
import base64

with open('temp.json', 'r') as f:
    data = json.load(f)
    councelList = data['data']['counselList']
    for counsel in councelList:
        for message in counsel['counsel']['conversation']:
            if message['role'] == 'user':
                for content in message['content']:
                    if content['type'] == 'image_url':
                        image = content['image_url']['url']
                        content['image_url']['url'] = 'image was here'

                        base64_str = image.split(',')[1]
                        image_data = base64.b64decode(base64_str)
                        
                        with open(f"output{counsel['id']}.jpg", 'wb') as photo:
                            
                            photo.write(image_data)
                        
    with open('temp_altered.json', 'w', encoding='utf-8') as f_altered:
        json.dump(data, f_altered, indent=4, ensure_ascii=False)
